In [3]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import os
from statsmodels.tsa.arima_model import ARIMA

import sys
sys.path.append("../")

selected_line = ['6450-10-0']

line = selected_line[0]
filename = './str(line) + '_temp.csv.gz'
df = pd.read_csv(filename, compression='gzip', sep=',')
df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
df.index = df['exact_time']

SyntaxError: invalid syntax (<ipython-input-3-9f8a6a3fd70f>, line 16)

In [ ]:
# ipython notebook --pylab=inline
%matplotlib agg

PLOT_ARIMA=True

# selected_line = ['6450-10-0']
test_samples = 1
for line in selected_line:    

    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
    df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-9-25', format = '%Y-%m-%d')

    df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
#     df = df.loc[(df['day'] > start_date) & (df['day'] < end_date) & (df['holiday'] != 1) & \
#                      ((df['weekday'] > 0) & (df['weekday'] < 5))]    

    img_dir = '../images/bussim/ARIMA/' + line +'_Links:' + str(max(df['link']))
    os.makedirs(img_dir,exist_ok=True)   
    
    frequency = '60min'            
    rolling_win = 3
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)

    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).mean()['time_link'].unstack()\
                .rolling(rolling_win).mean().transform(lambda x: x.fillna(method='ffill')).dropna().unstack().reset_index()

    X_Temp.columns = ['link', 'exact_time','time_link_mean']
    X_Temp.index = X_Temp['exact_time']            
    X_Temp['day_minute'] = pd.DatetimeIndex(X_Temp['exact_time']).hour*60 + pd.DatetimeIndex(X_Temp['exact_time']).minute

    for link in range(0, max(X_Temp['link']+1)):        
        if X_Temp.loc[X_Temp['link'] == link].empty:
            print("Not data of Link: " + str(link))
        else:        
            predictions = []
            measured = []
            for day_minute in X_Temp['day_minute'].unique():                    
                train = X_Temp[(X_Temp['day_minute'] == day_minute) & (X_Temp['link'] == link)]['time_link_mean'][:-test_samples]
                test = X_Temp[(X_Temp['day_minute'] == day_minute) & (X_Temp['link'] == link)]['time_link_mean'][-test_samples:]

                if (len(train) < 5):
                    predictions.append(np.mean(train.values))
                    measured.append(test.values)
                else:
                    model = ARIMA(train.values, order=(1,0,0))
                    model_fit = model.fit(disp=0)
                    output = model_fit.forecast()
                    predictions.append(output[0])
                    measured.append(test.values)

            fig, (ax, ax1) = plt.subplots(ncols=2, constrained_layout=True, figsize=(8,4))
            ax.plot(np.array(predictions))
            ax.plot(np.array(measured))
            ax.set_ylabel('Mean Travel Time')
            ax.set_xlabel('No. Samples | " Time line (Hours)')
            ax.set_title('Line:'+str(line) + ' | Link:' +str(link))
            ax.legend(['Predicted', 'Measured'])

            if (line == '6450-10-1') | (line == '8700-10-1') | (line == '2290-10-1') | \
                    (line == '477P-10-0') | (line == '3301-10-0') | (line == '477P-10-1') |\
                    (line == '351F-10-0'):
                ax1.boxplot(measured/predictions[0])
            else:
                ax1.boxplot(np.array(measured)/np.array(predictions))

            ax1.set_title('Predicted/Measured')
            ax1.set_ylabel('accuracy')
            ax1.set_xticklabels(labels=[link])

            file_name_fig = img_dir +'/' + 'Link:' + str(link) + '.png'
            fig.savefig(file_name_fig)
